In [6]:
import pandas as pd
import numpy as np
import json
import copy
import uuid

In [7]:
#Sellan data
source_file_statements = "source_data/DDB Sellan Coding source.xlsx"
source_file_actants = "source_data/DDB Mockup ACTANTS (entities Person, Concepts, Locations, Objects from Sellan).xlsx"

In [9]:
statement_template = {
    "id": "",
    "_source_id": "", # e.g. T3-1-01-013 if applicable
    "actionId": "", # A0090 (saw)
    "certainty": 0,
    "epistemicLevel": 0,
    "meta": "",
    "modality": 0,
    "tree": {},
    "note": "",
    "resources": [],
    "tags": [],
    "territoryId": "",
    "text": "David, strong friend of Peter, saw Adam in Brno."
    #"editor":"",
    #"primary_reference_part": "",
    #"secondary_reference_id": "",
    #"secondary_reference_part": "",
    #"secondary_reference_id": ""
}

node_tree_root_template = {
    "id": "",
     "_label": "",  
    "type": "root",
    "actants": [],
    "props": []   
}

has_property_template = {  # A:has property
     "id": "",
     "_label": "A: has property",
    "type": "property",
    "epistemicLevel": 0,
    "actants": [],
    "props": [],
    "resources": [],
    "tags": [],
    "territoryId": "",
    "modality": 0,
    "certainty": 0,
    "note": "",
    "meta": ""
}

actant_template = {  # entity
     "id": "",
     "_label": "", #P Person
    "type": "",  # subject,  actant1, actant2
    "props": []  
}


In [38]:
#read files
df_s = pd.read_excel(source_file_statements)
df_a = pd.read_excel(source_file_actants)


#clean statement dataframe
df_s = df_s.replace(np.nan, '', regex=True)  #rempve nans
df_s = df_s.astype(str) #cast all columns as string

#generate uuid for actants
def gen_uuid(row):
    return str(uuid.uuid4())
    
df_a['uuid'] = df_a.apply(lambda row: gen_uuid(row),axis=1) 

In [11]:
#creating dataset of territories
#in present import - the territory is just simply text_part_id



# - territories are like folders, they are nested in each other
# - each statement can be in exactly one territory 
#for key, row in df_s.iterrows():
#    print(row['id']

In [28]:
def create_object(template_dict):
    o = copy.deepcopy(template_dict)
    o['id'] = str(uuid.uuid4())
    return o

def create_statement(source_id=''):
    o = copy.deepcopy(statement_template)
    o['_source_id'] = source_id
    o['id'] = str(uuid.uuid4())
    return o

def create_has_property():
    o = copy.deepcopy(has_property_template)
    o['id'] = str(uuid.uuid4())
    return o


#simple
def create_actant(atype = 'subject', source_id = ''):
    o = copy.deepcopy(actant_template)
    #search for actant in master dataframe  df_a
    search =  df_a.loc[df_a['codeid'] == source_id]
    uuid = search['uuid']

    if uuid.empty:
        return False
    
    o['type'] = atype
    o['id'] = str(uuid.item())
    
    return o

#works with multiples
def create_actant_multiple(atype = 'subject', source_id = ''):
    
    actanct_ids = source_id.split("#")
    actancts = []
    
    for item in actanct_ids:
        o = copy.deepcopy(actant_template)
        search =  df_a.loc[df_a['codeid'] == item.strip()]  #search for actant in master dataframe  df_a
        uuid = search['uuid']
        if uuid.empty:
            return False

        o['_label'] = item.strip()
        o['type'] = atype
        o['id'] = str(uuid.item())
        actancts.append(o)
    
    return actancts




def returnTerritoryId(text_part):
    return text_part

In [27]:
str(uuid.uuid4())

'd411e27f-d5a7-4aed-9858-9d9f3c66e34f'

In [10]:
s = create_statement('aaaaaaa')
print(s)

a = create_actant('subject','P0004')
print(a)



{'uuid': 'd95ab859-4f33-4db0-a7df-92001d00baaa', '_source_id': 'source_id', 'actionId': '', 'certainty': '', 'epistemicLevel': '', 'meta': '', 'modality': '', 'tree': {}, 'note': '', 'resources': [], 'tags': [], 'territory': '', 'text': 'David, strong friend of Peter, saw Adam in Brno.', 'editor': '', 'primary_reference_part': '', 'secondary_reference_id': '', 'secondary_reference_part': ''}
{'uuid': '67895c30-baee-4841-abbd-b43a1c0635a8', '_label': '', 'type': 'subject', 'props': []}


In [39]:
df_s.columns

Index(['Unnamed: 0', 'id', 'parent_id', 'parent_id_old',
       'epistemological_level', 'text', 'subject_text', 'id_subject',
       'subject', 'modifier_subject\n', 'action_or_relation_text', 'modality',
       'id_action_or_relation', 'action_or_relation',
       'action_or_relation_english', 'modifier_action_or_relation',
       'id_actant1', 'actant1\n', 'modifier_actant1', 'id_actant2', 'actant2',
       'modifier_actant2', 'dissidence_or_compliance_level', 'tags',
       'time_text', 'timerelation1_number', 'timerelation1_time_unit',
       'timerelation1_type', 'timerelation1_target_id', 'timerelation2_number',
       'timerelation2_time_unit', 'timerelation2_type',
       'timerelation2_target_id', 'start_date_earliest\n', 'start_date_latest',
       'end_date_earliest', 'end_date_latest', 'time_fuzzy', 'time_note\n',
       'duration_text', 'duration_number', 'duration_time_unit',
       'duration_fuzzy', 'frequency_text', 'frequency_number',
       'frequency_time_unit', 'fr

In [64]:
statements = []  # list objectu statments

#idiot simple structure which ignores almost any complexity
#  1) does not work with multiples
#  2) needs C or P in id_subject


##############################################################
i = 0
for key, row in df_s.iterrows():
    i += 1
    if (i>5):
        #break
        pass
    
    #basic 'inteligence'
    if ('P' not in row['id_subject'] and 'C' not in row['id_subject']):
        print ('Skipping '+  row['id'] + " : strange id_subject "+row['id_subject'])
        continue
    
    
    #cretate statement object
    s = create_statement(statement_template)
    s['_source_id'] = row['id']
    s['actionId'] = row['id_action_or_relation']
    
    elvl = row['epistemological_level'].split('-')[0]
    if (len(elvl) > 0):
        s['epistemicLevel'] = int(elvl) #konverze 2-interpretative na 2   
    else:
        s['epistemicLevel'] =  1
    
    s['modality'] = row['modality']
    s['certainty'] =  row['certainty']
    s['text'] = row['text'].strip()
    s['territoryId'] = row['text_part_id']  #nevraci uuid, ale label "T3-1-64", nicmene by to nemelo vadit
    
    #s["primary_reference_part"] = row["primary_reference_part"]
    #s["secondary_reference_id"] = row["secondary_reference_id"]
    #s["secondary_reference_part"] = row["secondary_reference_part"]
    #s["secondary_reference_id"] = row["secondary_reference_id"]
    
    
    #create root action object
    ra = create_object(node_tree_root_template)
    #print(ra['uuid'])
    
    #create subject
    su = create_actant_multiple('subject',row['id_subject'])
    if su:
        ra['actants'].extend(su)
    else:
        print('Subject '+row['id_subject']+" not recognized.")
    
    
    #create actant1
    if (row['id_actant1']):    
        a1 = create_actant_multiple('actant1',row['id_actant1'])
        if a1:
            ra['actants'].extend(a1)
    else:
        #print('Empty a1')
        pass
    
    #create actant2
    if (row['id_actant2']):    
        a2 = create_actant_multiple('actant2',row['id_actant2'])
        if (a2):
            ra['actants'].extend(a2)
    else:
        #print('Empty a2')
        pass
    
    #specialni pripady,  konstrukce has_properties vazeb u lokaci
    ############################################################
    #C0223  location: where
    #C9224	location: to	CONCEPT
    #C9225	location: from	CONCEPT  : v Sellanu neexistuje

    
    
    #id_location  - understood as location_where   C0223
    # : nove has propertion, ktere ma jako subjekt root akci
    #                              (ve techto datech ale neni subjekt zadan)
    if ("L" in str(row['id_location']) or "C" in str(row['id_location'])):
        has_property = create_has_property()
        
        
        if ("[" in row['id_location']):
            has_property['epistemicLevel'] = 3
            row['id_location'] = row['id_location'].replace("[","")
            row['id_location'] = row['id_location'].replace("]","")
                 
        #l_s = ... :  aby tu nebyla rekurze, tak se actant suject ingoruje 
        
        l_a1 = create_actant('actant1', "C0223")    
        l_a2 = create_actant_multiple('actant2', row['id_location'])
        
        if (l_a1 and l_a2):
            has_property['actants'].extend([l_a1])
            has_property['actants'].extend(l_a2)
            ra['props'].extend(has_property)
            print("Success in creating location_where has_property. "+ row['id'])
        else:
            print("Failing in creating location_where has_property. "+ row['id']+" : " + row['id_location'])
    
    #id_location)_to - understood as location_where   C0224
    # : nove has propertion, ktere ma jako subjekt concept v id_actant2 root akci
    # kdyz je v id_actant2 C jako trest
    if ("C" in row['id_actant2'] and row['id_location_to']):
        has_property = create_has_property()
        
        #l_s = ... :  aby tu nebyla rekurze, tak se actant suject ingoruje 
        
        l_a1 = create_actant('actant1', "C9224")    
        l_a2 = create_actant_multiple('actant2', row['id_location_to'])
        
        if (l_a1 and l_a2):
            has_property['actants'].extend([l_a1])
            has_property['actants'].extend(l_a2)
            a2[0]['props'].extend(has_property)
            print("Success in creating location_to has_property. "+ row['id'])
        else:
            print("Failing in creating location_to has_property. "+ row['id'] + " : " + row['id_location_to'])
            

    s['tree'] = ra
    statements.append(s)

Skipping T3-1-00-001 : strange id_subject T3-1
Skipping T3-1-00-002 : strange id_subject T3-1
Success in creating location_where has_property. T3-1-01-009
Success in creating location_to has_property. T3-1-01-014
Success in creating location_to has_property. T3-1-01-015
Success in creating location_to has_property. T3-1-01-016
Success in creating location_to has_property. T3-1-01-017
Success in creating location_to has_property. T3-1-01-018
Success in creating location_to has_property. T3-1-01-019
Success in creating location_to has_property. T3-1-01-020
Success in creating location_to has_property. T3-1-01-021
Success in creating location_to has_property. T3-1-02-032
Success in creating location_to has_property. T3-1-02-034
Success in creating location_to has_property. T3-1-03-039
Success in creating location_to has_property. T3-1-03-040
Success in creating location_to has_property. T3-1-03-041
Success in creating location_to has_property. T3-1-03-042
Success in creating location_to h

Success in creating location_to has_property. T3-1-46-005
Success in creating location_to has_property. T3-1-46-006
Success in creating location_to has_property. T3-1-46-007
Success in creating location_to has_property. T3-1-46-008
Success in creating location_to has_property. T3-1-46-009
Success in creating location_to has_property. T3-1-46-010
Success in creating location_to has_property. T3-1-46-011
Success in creating location_to has_property. T3-1-46-012
Success in creating location_to has_property. T3-1-461-003
Success in creating location_to has_property. T3-1-461-004
Success in creating location_to has_property. T3-1-461-005
Success in creating location_to has_property. T3-1-47-003
Success in creating location_to has_property. T3-1-48-004
Success in creating location_to has_property. T3-1-48-005
Success in creating location_to has_property. T3-1-48-006
Success in creating location_to has_property. T3-1-48-007
Success in creating location_to has_property. T3-1-48-008
Success in 

Success in creating location_to has_property. T3-1-92-003
Success in creating location_to has_property. T3-1-92-004
Success in creating location_to has_property. T3-1-93-003
Success in creating location_to has_property. T3-1-93-004
Success in creating location_to has_property. T3-1-94-003
Success in creating location_to has_property. T3-1-94-004
Success in creating location_to has_property. T3-1-95-003
Success in creating location_to has_property. T3-1-95-004
Success in creating location_to has_property. T3-1-95-005
Success in creating location_to has_property. T3-1-95-006
Success in creating location_to has_property. T3-1-95-007
Success in creating location_to has_property. T3-1-95-008
Success in creating location_to has_property. T3-1-95-009
Success in creating location_to has_property. T3-1-95-010
Success in creating location_to has_property. T3-1-96-003
Success in creating location_to has_property. T3-1-96-004
Success in creating location_to has_property. T3-1-96-005
Success in cre

Success in creating location_to has_property. T3-1-9929-018
Success in creating location_to has_property. T3-1-9929-019
Success in creating location_to has_property. T3-1-9929-020
Success in creating location_to has_property. T3-1-9929-021
Success in creating location_to has_property. T3-1-9929-022
Skipping T3-1-9929-023 : strange id_subject T3-1-9929-014
Skipping T3-1-9929-024 : strange id_subject T3-1-9929-015
Skipping T3-1-9929-025 : strange id_subject T3-1-9929-016
Skipping T3-1-9929-026 : strange id_subject T3-1-9929-017
Skipping T3-1-9929-027 : strange id_subject T3-1-9929-018
Skipping T3-1-9929-028 : strange id_subject T3-1-9929-019
Skipping T3-1-9929-029 : strange id_subject T3-1-9929-020
Skipping T3-1-9929-030 : strange id_subject T3-1-9929-021
Skipping T3-1-9929-031 : strange id_subject T3-1-9929-022
Success in creating location_to has_property. T3-1-9930-005
Success in creating location_to has_property. T3-1-9930-006
Success in creating location_to has_property. T3-1-9930-00

Success in creating location_to has_property. T3-1-9952-008
Success in creating location_to has_property. T3-1-9952-009
Success in creating location_to has_property. T3-1-9952-010
Skipping T3-1-9952-011 : strange id_subject T3-1-9952-008
Skipping T3-1-9952-012 : strange id_subject T3-1-9952-009
Skipping T3-1-9952-013 : strange id_subject T3-1-9952-010
Success in creating location_where has_property. T3-1-9953-006
Success in creating location_where has_property. T3-1-9953-009
Failing in creating location_to has_property. T3-1-9953-015 : NS
Success in creating location_to has_property. T3-1-9953-016
Success in creating location_to has_property. T3-1-9953-017
Success in creating location_to has_property. T3-1-9953-018
Success in creating location_to has_property. T3-1-9953-019
Success in creating location_to has_property. T3-1-9953-020
Success in creating location_to has_property. T3-1-9953-021
Success in creating location_to has_property. T3-1-9953-022
Success in creating location_to has

In [65]:
#df_a.to_excel('import_data/sellan/actants.xlsx')
#df_a.to_csv('import_data/sellan/actants.csv')
df_a.to_json('import_data/sellan/actants.json', orient='records')


statements_json = json.dumps(statements)
#print(statements_json)

with open('import_data/sellan/statements.json', 'w') as file:
    file.write(statements_json)